In [1]:
using Revise
using IterativeSolvers
using LinearAlgebra

using Pkg
Pkg.activate("..")
using FSI

using LinearAlgebra

In [2]:
# construct a random block system

# block size
m = 10
n = 8
p = 6
q = 4

# operators
A = rand(m,m); A⁻¹ = inv(A)
B₁ᵀ = rand(m,n)
B₂ = rand(n,m)
M = rand(p,p)
G₁ᵀ = rand(p,q)
G₂ = rand(q,p)
T₁ᵀ = rand(p,n)
T₂ = rand(n,p)

# zero matrix
Omp = zeros(m,p)
Omq = zeros(m,q)
Onn = zeros(n,n)
Onq = zeros(n,q)
Opm = zeros(p,m)
Oqm = zeros(q,m)
Oqn = zeros(q,n)
Oqq = zeros(q,q)

# fictitious fluid
ρb = 2.0
Mf = 1.0/ρb*M

# saddle point system
S = [A B₁ᵀ -B₁ᵀ*T₂*Mf Omq; B₂ Onn -T₂ Onq; Opm -T₁ᵀ M G₁ᵀ; Oqm Oqn G₂ Oqq]

# rhs
rċ = rand(m)
rf = rand(n)
ru̇ = rand(p)
rλ = rand(q)

b = [rċ;rf;ru̇;rλ];
b1 = [rċ;rf]
b2 = [ru̇;rλ];

In [3]:
# construct saddle point system
ċ = zeros(m)
f = zeros(n)
u̇ = zeros(p)
λ = zeros(q)
@time St = SaddleSystem2d((ċ, f, u̇, λ), (A⁻¹, B₁ᵀ, B₂),
                  (M, G₁ᵀ, G₂), (x->T₁ᵀ*x, x->T₂*x);
                  ρb=ρb)

  1.365374 seconds (5.41 M allocations: 272.556 MiB, 5.89% gc time)


Saddle system with 8 constraints on fluid and 4 constraints on 2d body
   Fluid state of type Array{Float64,1}
   Fluid force of type Array{Float64,1}
   Body state of type Array{Float64,1}
   Joint force of type Array{Float64,1}


In [4]:
# # compared to 1d system
# S2 = [A B₁ᵀ Omp Omq; B₂ Onn -T₂ Onq; Opm T₁ᵀ M G₁ᵀ; Oqm Oqn G₂ Oqq]
# S2\b

In [5]:
# Julia solver
@time x_theory = S\b

  1.778018 seconds (2.28 M allocations: 107.847 MiB, 2.02% gc time)


28-element Array{Float64,1}:
  0.0039036796749947118
 -0.03789876550319593  
 -0.09394525559917069  
  0.03598870440478218  
 -0.060689547126037385 
  1.6665948844004324   
 -0.433981356394322    
 -0.2888528820041774   
  0.7340080209100269   
 -0.9017174567512328   
  2.4002230851830513   
  1.9103943436452597   
 -0.914104553623504    
  ⋮                    
  1.2615262094857458   
 -1.0813056077086067   
  1.0486721686054372   
  1.1137005144975225   
 -0.20116544583528245  
 -0.3472267988963471   
 -0.2994776874661479   
 -0.34369022063623983  
  0.7840711402199902   
  1.2669272191631429   
  2.121083562230898    
 -0.8019542646356641   

In [6]:
# my own solver
bt = (rċ, rf, ru̇, rλ);
aa,bb,cc,dd = St\bt
@time x_compute = [aa;bb;cc;dd]

  0.000005 seconds (6 allocations: 544 bytes)


28-element Array{Float64,1}:
  0.0039036796749885028
 -0.037898765503191933 
 -0.09394525559916467  
  0.03598870440478974  
 -0.06068954712604313  
  1.6665948844004361   
 -0.4339813563943198   
 -0.28885288200417936  
  0.7340080209100195   
 -0.9017174567512319   
  2.4002230851830566   
  1.9103943436452635   
 -0.9141045536234949   
  ⋮                    
  1.2615262094857553   
 -1.0813056077086034   
  1.0486721686054414   
  1.1137005144975258   
 -0.201165445835289    
 -0.347226798896357    
 -0.2994776874661399   
 -0.34369022063623295  
  0.7840711402199982   
  1.2669272191631415   
  2.1210835622308952   
 -0.8019542646356487   

In [7]:
norm(x_theory-x_compute)

4.6740201035483336e-14

### check step 1

In [8]:
[A B₁ᵀ;B₂ Onn]\b1
# norm([A B₁ᵀ;B₂ Onn]*x1 - [rċ;rf])

18-element Array{Float64,1}:
  1.2738615307937844 
  0.35078914998157507
 -1.0467236715243367 
 -0.35074864877244954
 -2.2992030699124815 
  1.282873689785597  
 -1.7229882849376157 
  0.7549639029604989 
  1.8235555092762477 
  0.3856847664229972 
 -0.401166029431642  
  3.3626066646336414 
 -2.112364689048015  
  2.592416506383522  
 -2.847040371417126  
 -2.507468117391492  
  2.241093866446113  
  0.3216235535935262 

In [9]:
using ImportMacros
@import ViscousFlow.SaddlePointSystems as Sad

Sf = Sad.SaddleSystem((ċ,f),(A⁻¹,B₁ᵀ,B₂),tol=1e-3,
            issymmetric=false,isposdef=true,store=true,precompile=true)
x1,x2 = Sf\(b1[1:m], b1[m+1:end])

([1.27386, 0.350789, -1.04672, -0.350749, -2.2992, 1.28287, -1.72299, 0.754964, 1.82356, 0.385685], [-0.401166, 3.36261, -2.11236, 2.59242, -2.84704, -2.50747, 2.24109, 0.321624])

### check step 2

In [10]:
x_theory[m+n+1:end]

10-element Array{Float64,1}:
  1.0486721686054372 
  1.1137005144975225 
 -0.20116544583528245
 -0.3472267988963471 
 -0.2994776874661479 
 -0.34369022063623983
  0.7840711402199902 
  1.2669272191631429 
  2.121083562230898  
 -0.8019542646356641 

In [11]:
[M-T₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂+T₁ᵀ*T₂*Mf G₁ᵀ;G₂ Oqq]\[b2[1:p]-T₁ᵀ*x2;b2[p+1:end]]

10-element Array{Float64,1}:
  0.8707500131889191 
  0.8117927177314024 
  0.08999065901623018
  0.29787354258491855
 -0.6014609431524688 
 -0.7966339942732612 
 -2.250989112846042  
  0.3131961889251749 
  0.4484922612288399 
 -0.5796774574899748 

In [12]:
x34 = gmres([M-T₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂+T₁ᵀ*T₂*Mf G₁ᵀ;G₂ Oqq], [b2[1:p]-T₁ᵀ*x2;b2[p+1:end]], tol=1e-3)

10-element Array{Float64,1}:
  0.8707500131889194 
  0.8117927177314017 
  0.08999065901623005
  0.2978735425849174 
 -0.6014609431524689 
 -0.7966339942732599 
 -2.2509891128460424 
  0.31319618892517515
  0.44849226122884145
 -0.5796774574899767 

In [13]:
x_compute[m+n+1:end]

10-element Array{Float64,1}:
  1.0486721686054414 
  1.1137005144975258 
 -0.201165445835289  
 -0.347226798896357  
 -0.2994776874661399 
 -0.34369022063623295
  0.7840711402199982 
  1.2669272191631415 
  2.1210835622308952 
 -0.8019542646356487 

In [14]:
cond([M-T₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂+T₁ᵀ*T₂*Mf G₁ᵀ;G₂ Oqq])

351.091070944088

### check step 3

In [15]:
x_theory[1:m+n]

18-element Array{Float64,1}:
  0.0039036796749947118
 -0.03789876550319593  
 -0.09394525559917069  
  0.03598870440478218  
 -0.060689547126037385 
  1.6665948844004324   
 -0.433981356394322    
 -0.2888528820041774   
  0.7340080209100269   
 -0.9017174567512328   
  2.4002230851830513   
  1.9103943436452597   
 -0.914104553623504    
  3.19585812742723     
 -1.710820528293717    
 -0.8588762994666257   
  1.2615262094857458   
 -1.0813056077086067   

In [16]:
ċ = x1 + A⁻¹*B₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂*x34[1:p]

10-element Array{Float64,1}:
  0.7919948864420929 
 -0.30480933902329244
 -0.6080731455007945 
 -0.0942486666878859 
 -0.16243597869183057
  0.7430442301718968 
 -0.45388282888590425
 -0.15303801054667943
  0.9496973393663847 
 -0.2912621590600807 

In [17]:
f = x2 - inv(B₂*A⁻¹*B₁ᵀ)*T₂*x34[1:p] + T₂*Mf*x34[1:p]

8-element Array{Float64,1}:
  0.44349082970085824
  0.8544395830030235 
 -0.854507418680524  
  0.5834486676703916 
 -0.17534716726414523
  0.4168668542868168 
  1.6137518026364692 
  0.3320164620624121 

In [18]:
x_compute[1:m+n]

18-element Array{Float64,1}:
  0.0039036796749885028
 -0.037898765503191933 
 -0.09394525559916467  
  0.03598870440478974  
 -0.06068954712604313  
  1.6665948844004361   
 -0.4339813563943198   
 -0.28885288200417936  
  0.7340080209100195   
 -0.9017174567512319   
  2.4002230851830566   
  1.9103943436452635   
 -0.9141045536234949   
  3.195858127427259    
 -1.7108205282937283   
 -0.8588762994666377   
  1.2615262094857553   
 -1.0813056077086034   